In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
from collections import Counter
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPool2D
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report
from sklearn.preprocessing import RobustScaler
from scipy import stats
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model
from google.colab import drive
drive.mount('/content/gdrive')

TIME_STEPS = 200
STEP = 40


Mounted at /content/gdrive


In [ ]:
register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 22, 10
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

<b><h1> Pre-Processing dei dati </h1></b> <br>


In [ ]:
def create_dataset(X, y, time_steps=1, step=1):
  TIME_STEPS = 200#isla
  STEP = 40
  Xs, ys = [], []
  for i in range(0, len(X)-time_steps, step):
    v = X.iloc[i: (i+time_steps)].values
    labels = y.iloc[i: i+time_steps]
    Xs.append(v)
    ys.append(stats.mode(labels)[0][0])
  return np.array(Xs), np.array(ys).reshape(-1, 1)

def plot_learningCurve(history, epochs):
  # Plot training & validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Validation Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
column_names = ['user_id', 'activity', 'timestamp', 'X', 'Y', 'Z']
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dataset/DatasetUniba.csv', header = None, names = column_names)
df.dropna(axis=0, how='any', inplace=True)

df

,user_id,activity,timestamp,X,Y,Z
0,1,BackwardFall,0,-2.120995,-7.684790,-0.116293
1,1,BackwardFall,5,-1.837475,-7.524046,-0.273713
2,1,BackwardFall,10,-1.528029,-7.397976,-0.263592
3,1,BackwardFall,15,-1.099379,-7.245015,-0.243442
4,1,BackwardFall,20,-0.651752,-7.112909,-0.147084
...,...,...,...,...,...,...
970347,19,Walking,15030,3.607614,3.775315,-1.813403
970348,19,Walking,15035,3.749106,4.399148,-2.120795
970349,19,Walking,15040,3.862242,5.141159,-2.582244
970350,19,Walking,15045,3.913796,6.209425,-3.008182


In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
df["activity"].loc[df.activity == 'BackwardFall'] = "Fall"
df["activity"].loc[df.activity == 'ForwardFall'] = "Fall"
df["activity"].loc[df.activity == 'LeftFall'] = "Fall"
df["activity"].loc[df.activity == 'RightFall'] = "Fall"
#df = df[df.activity != "Sitting"] #Rimuovendo il commento iniziale si eseguirà il classificatore no sitting
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970352 entries, 0 to 970351
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    970352 non-null  int64  
 1   activity   970352 non-null  object 
 2   timestamp  970352 non-null  int64  
 3   X          970352 non-null  float64
 4   Y          970352 non-null  float64
 5   Z          970352 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 44.4+ MB


In [ ]:
df.describe()

,user_id,timestamp,X,Y,Z
count,970352.000000,970352.000000,970352.000000,970352.000000,970352.000000
mean,9.652475,7486.316867,0.406984,-0.656867,-0.275370
std,5.530966,4343.144385,4.259469,7.203429,4.695829
min,1.000000,0.000000,-56.221443,-61.331863,-58.738260
25%,5.000000,3730.000000,-2.204359,-3.413708,-3.185539
50%,9.000000,7460.000000,0.432485,-0.275161,-0.170318
75%,14.000000,11220.000000,2.824348,1.845646,1.931052
max,19.000000,15115.000000,57.871265,61.735350,52.757180


In [ ]:
print(df["user_id"].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
print(df[df["user_id"]==1])

       user_id activity  timestamp         X         Y         Z
0            1     Fall          0 -2.120995 -7.684790 -0.116293
1            1     Fall          5 -1.837475 -7.524046 -0.273713
2            1     Fall         10 -1.528029 -7.397976 -0.263592
3            1     Fall         15 -1.099379 -7.245015 -0.243442
4            1     Fall         20 -0.651752 -7.112909 -0.147084
...        ...      ...        ...       ...       ...       ...
54192        1  Walking      11215 -1.589849 -7.083926  1.843804
54193        1  Walking      11220 -1.357899 -7.042838  1.565295
54194        1  Walking      11225 -1.092040 -7.130431  1.361393
54195        1  Walking      11230 -0.852474 -7.201789  1.099617
54196        1  Walking      11235 -0.564648 -7.313608  0.854749

[54197 rows x 6 columns]


<b><h1> LOO </h1></b> <br>
Definizione del classificatore Convolutional Neural Network e stampa delle metriche della classificazione.

In [ ]:
from sklearn.metrics import *

In [ ]:
k = len(df["user_id"].unique())
indici = np.arange(start=1, stop=k+1)
print(indici)

19
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Parte dedita ad addestramento modello

In [ ]:
df1 = pd.DataFrame()
principal = indici
f1=[]
acc=[]

for i in principal:
  df1 = pd.DataFrame()
  indici = np.delete(indici, np.where(indici == i))

  for k in indici:
    df1 = df1.append(df[df['user_id'] == int(k)], ignore_index=True)

  print("UTENTE: %d" % i)
  ##
  df_train = df1
  #df_test = prova_dataset

  scale_columns = ['X', 'Y', 'Z']
  scaler = RobustScaler()
  scaler = scaler.fit(df[scale_columns])

  df_train.loc[:, scale_columns] = scaler.transform(df_train[scale_columns].to_numpy())
  #df_test.loc[:, scale_columns] = scaler.transform(df_test[scale_columns].to_numpy())

  X_train, y_train = create_dataset(df_train[['X', 'Y', 'Z']], df_train.activity, TIME_STEPS, STEP)
  #X_test, y_test = create_dataset(df_test[['X', 'Y', 'Z']], df_train.activity, TIME_STEPS, STEP)

  enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
  enc = enc.fit(y_train)

  y_train = enc.transform(y_train)
  #y_test = enc.transform(y_test)

  X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
  #X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
  print(X_train.shape)

  ##
  """Conv2D"""
  model = Sequential()
  model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape = X_train[0].shape))
  model.add(Dropout(0.5))
  model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
  model.add(Dropout(0.5))
  model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
  model.add(Dropout(0.5))
  """"""

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='softmax'))
  model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy', f1_m])

  model.fit(X_train, y_train, batch_size = 512, epochs = 10, validation_split = 0.2, verbose=1)

  #print(model.evaluate(X_train,y_train))
  #print(model.evaluate(X_test,y_test))

  #plot_learningCurve(history, 6)
  '''

  rounded_labels=np.argmax(y_train, axis=1)
  y_pred = np.argmax(model.predict(X_train), axis=-1)

  acc.append(accuracy_score(rounded_labels, y_pred))
  f1.append(f1_score(rounded_labels, y_pred, average='macro'))

  print(classification_report(rounded_labels, y_pred, digits=4))
  '''
  print(model.evaluate(X_train,y_train))
  #print(model.evaluate(X_test,y_test))

  #plot_learningCurve(history, 6)

  rounded_labels=np.argmax(y_train, axis=1)
  y_pred = np.argmax(model.predict(X_train), axis=-1)

  acc.append(accuracy_score(rounded_labels, y_pred))
  f1.append(f1_score(rounded_labels, y_pred, average='macro'))

  print(classification_report(rounded_labels, y_pred, digits=4))

  print(np.mean(acc))
  print(np.mean(f1))

  indici = principal

<ipython-input-19-6d861c7478c8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df[df['user_id'] == int(k)], ignore_index=True)


UTENTE: 1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
<ipython-input-13-1d095abc8dfe>:9: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
<ipython-input-13-1d095abc8dfe>:9: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  ys.append(stats.mode(labels)[0][0])
/usr/local/lib/python3.10

(22899, 200, 3, 1)
Epoch 1/10
36/36 [==============================] - 71s 2s/step - loss: 1.0905 - accuracy: 0.5739 - f1_m: 0.4816 - val_loss: 0.9648 - val_accuracy: 0.5079 - val_f1_m: 0.4570
Epoch 2/10
36/36 [==============================] - 73s 2s/step - loss: 0.5424 - accuracy: 0.8041 - f1_m: 0.7803 - val_loss: 0.5794 - val_accuracy: 0.8611 - val_f1_m: 0.8272
Epoch 3/10
36/36 [==============================] - 68s 2s/step - loss: 0.3959 - accuracy: 0.8522 - f1_m: 0.8436 - val_loss: 0.5233 - val_accuracy: 0.8729 - val_f1_m: 0.8677
Epoch 4/10
36/36 [==============================] - 66s 2s/step - loss: 0.3352 - accuracy: 0.8752 - f1_m: 0.8703 - val_loss: 0.4493 - val_accuracy: 0.8712 - val_f1_m: 0.8695
Epoch 5/10
36/36 [==============================] - 66s 2s/step - loss: 0.3014 - accuracy: 0.8877 - f1_m: 0.8857 - val_loss: 0.4421 - val_accuracy: 0.8666 - val_f1_m: 0.8673
Epoch 6/10
36/36 [==============================] - 77s 2s/step - loss: 0.2794 - accuracy: 0.9015 - f1_m: 0.897

<ipython-input-19-6d861c7478c8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df[df['user_id'] == int(k)], ignore_index=True)


...

(23047, 200, 3, 1)
Epoch 1/10
37/37 [==============================] - 65s 2s/step - loss: 1.0203 - accuracy: 0.6306 - f1_m: 0.5224 - val_loss: 0.9064 - val_accuracy: 0.6208 - val_f1_m: 0.5015
Epoch 2/10
37/37 [==============================] - 67s 2s/step - loss: 0.4751 - accuracy: 0.8208 - f1_m: 0.7985 - val_loss: 0.6732 - val_accuracy: 0.7800 - val_f1_m: 0.6901
Epoch 3/10
37/37 [==============================] - 65s 2s/step - loss: 0.3731 - accuracy: 0.8579 - f1_m: 0.8527 - val_loss: 1.1066 - val_accuracy: 0.6954 - val_f1_m: 0.6732
Epoch 4/10
37/37 [==============================] - 65s 2s/step - loss: 0.3270 - accuracy: 0.8793 - f1_m: 0.8777 - val_loss: 1.4820 - val_accuracy: 0.6655 - val_f1_m: 0.5990
Epoch 5/10
37/37 [==============================] - 67s 2s/step - loss: 0.2843 - accuracy: 0.8988 - f1_m: 0.8927 - val_loss: 1.7403 - val_accuracy: 0.6683 - val_f1_m: 0.6019
Epoch 6/10
37/37 [==============================] - 64s 2s/step - loss: 0.2838 - accuracy: 0.8940 - f1_m: 0.893

<ipython-input-19-6d861c7478c8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df[df['user_id'] == int(k)], ignore_index=True)


UTENTE: 19


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
<ipython-input-13-1d095abc8dfe>:9: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
<ipython-input-13-1d095abc8dfe>:9: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  ys.append(stats.mode(labels)[0][0])
/usr/local/lib/python3.10

(23047, 200, 3, 1)
Epoch 1/10
37/37 [==============================] - 66s 2s/step - loss: 0.9978 - accuracy: 0.6140 - f1_m: 0.5449 - val_loss: 0.9929 - val_accuracy: 0.5134 - val_f1_m: 0.4262
Epoch 2/10
37/37 [==============================] - 66s 2s/step - loss: 0.5527 - accuracy: 0.7958 - f1_m: 0.7723 - val_loss: 1.1312 - val_accuracy: 0.6421 - val_f1_m: 0.5751
Epoch 3/10
37/37 [==============================] - 67s 2s/step - loss: 0.4226 - accuracy: 0.8436 - f1_m: 0.8370 - val_loss: 0.8510 - val_accuracy: 0.7603 - val_f1_m: 0.6841
Epoch 4/10
37/37 [==============================] - 68s 2s/step - loss: 0.3483 - accuracy: 0.8689 - f1_m: 0.8687 - val_loss: 1.0923 - val_accuracy: 0.7061 - val_f1_m: 0.6328
Epoch 5/10
37/37 [==============================] - 66s 2s/step - loss: 0.3031 - accuracy: 0.8869 - f1_m: 0.8768 - val_loss: 1.1379 - val_accuracy: 0.7705 - val_f1_m: 0.7430
Epoch 6/10
37/37 [==============================] - 66s 2s/step - loss: 0.2782 - accuracy: 0.8955 - f1_m: 0.896

In [ ]:

  print(model.evaluate(X_train,y_train))
  #print(model.evaluate(X_test,y_test))

  #plot_learningCurve(history, 6)

  rounded_labels=np.argmax(y_train, axis=1)
  y_pred = np.argmax(model.predict(X_train), axis=-1)

  acc.append(accuracy_score(rounded_labels, y_pred))
  f1.append(f1_score(rounded_labels, y_pred, average='macro'))

  print(classification_report(rounded_labels, y_pred, digits=4))

  print(np.mean(acc))
  print(np.mean(f1))

721/721 [==============================] - 17s 23ms/step - loss: 0.5060 - accuracy: 0.9212 - f1_m: 0.9202
[0.5060463547706604, 0.9212045073509216, 0.9201885461807251]
721/721 [==============================] - 24s 33ms/step
              precision    recall  f1-score   support

           0     0.9052    0.9713    0.9371     11250
           1     0.9945    0.9573    0.9755      2999
           2     0.9979    0.9376    0.9668      2998
           3     0.8154    0.7007    0.7537      2793
           4     0.9279    0.8863    0.9066      3007

    accuracy                         0.9212     23047
   macro avg     0.9282    0.8906    0.9080     23047
weighted avg     0.9210    0.9212    0.9198     23047

0.9150307199035946
0.9009571661389011


In [ ]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 200, 3, 64)        320       
                                                                 
 dropout_72 (Dropout)        (None, 200, 3, 64)        0         
                                                                 
 conv2d_55 (Conv2D)          (None, 200, 3, 32)        8224      
                                                                 
 dropout_73 (Dropout)        (None, 200, 3, 32)        0         
                                                                 
 conv2d_56 (Conv2D)          (None, 200, 3, 16)        2064      
                                                                 
 dropout_74 (Dropout)        (None, 200, 3, 16)        0         
                                                                 
 flatten_18 (Flatten)        (None, 9600)            

In [ ]:
model.save('/content/gdrive/MyDrive/T5_CNN.h5')

In [ ]:
model = load_model('/content/gdrive/MyDrive/T5_CNN.h5' , custom_objects={"f1_m": f1_m, "precision_m" : precision_m, "recall_m" : recall_m})

In [ ]:
def create_dataset2(X, time_steps=1, step=1):
  TIME_STEPS = 200#
  STEP = 40
  Xs = []
  for i in range(0, len(X)-time_steps, step):
    v = X.iloc[i: (i+time_steps)].values
    Xs.append(v)
  return np.array(Xs)

funzione a cui passo nome file csv di un utente e che da in pasto i dati sei sensori per ottenere le predizioni per ogni riga

In [ ]:
def predict(file_name):
  scaler = RobustScaler()
  scale_columns = ['X', 'Y', 'Z']
  csv = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/csv_estratti/%s' % file_name, index_col=None).drop(columns=['Unnamed: 0'])
  print(csv)
  scaler.fit(csv[scale_columns])#X,Y,Z
  csv.loc[:, scale_columns] = scaler.transform(csv[scale_columns].to_numpy())

  X_train_m = create_dataset2(csv[['X', 'Y', 'Z']], TIME_STEPS, 1)

  X_train_m = X_train_m.reshape(X_train_m.shape[0], X_train_m.shape[1], X_train_m.shape[2], 1)

  return model.predict(X_train_m),  np.argmax(model.predict(X_train_m), axis=-1)

modifica il vettore ottenuto dalla predizione, da essere formato da soli numeri che indicano le classi di appartenenza, vengono tradotte nelle corrispondenti attività che dipendono dal dataset

In [ ]:
def translate(vett):
  print(vett)
  translated_vett = []

  for i in range(0,len(vett)):
    if(vett[i]==0):
      translated_vett.append('Sitting')
    elif(vett[i]==1):
      translated_vett.append('Jogging')
    elif(vett[i]==2):
      translated_vett.append('Fall')
    elif(vett[i]==3):
      translated_vett.append('Hopping')
    elif(vett[i]==4):
      translated_vett.append('Walking')

  return translated_vett

stampa una resoconto delle righe incontrate, cioè la somma di ogni colonna(attività)

In [ ]:
def printer(predicted_vett):
  df_activity = pd.DataFrame(predicted_vett)
  df_activity.columns=["Sitting", "Jogging", "Fall", "Hopping", "Walking"]
  print(df_activity)
  print(np.sum(df_activity['Sitting']))
  print(np.sum(df_activity['Jogging']))
  print(np.sum(df_activity['Fall']))
  print(np.sum(df_activity['Hopping']))
  print(np.sum(df_activity['Walking']))

  print(np.sum(df_activity['Sitting']+df_activity['Jogging']+df_activity['Fall']+df_activity['Hopping']+df_activity['Walking']))

questa funzione prende in input il nome del file csv e il vettore delle attività tradotto e lo inserisce in nuova colonna del file csv di partenza dell utente stesso. Questo viene poi salvato

In [ ]:
def save2(file_name, predicted_vett):
  csv = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/csv_estratti/%s' % file_name, index_col=None).drop(columns=['Unnamed: 0'])
  predicted_user = csv
  predicted_user = predicted_user.drop(predicted_user.index[len(predicted_user)-200:len(predicted_user)])

  predicted_user['PredictedActivity']=predicted_vett
  w1,w2 = file_name.split('.')

  if(not os.path.isdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B')):
    os.mkdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B')

  if(not os.path.isdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/completi')):
    os.mkdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1A/completi')

  predicted_user.to_csv('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/completi/%s_Predicted.csv' % w1)
  return predicted_user

Questa funzione crea un array con tanti elementi quante sono le schermate a cui l'utente ha partecipato. in particolare ogni elemento definisce l'ultima riga del file per schermata

In [ ]:
def find_activity(predicted_user):
  interval = []
  for row in range(0,len(predicted_user)-1):
    if(predicted_user['NameActivity'].iloc[row]!=predicted_user['NameActivity'].iloc[row+1]):
      interval.append(row)
  return interval

funzione che calcola percentuali di attività svolte rispetto al totale

In [ ]:
def percentage(row, resoconto):
  falling_sum = np.sum(resoconto['Fall'])
  hopping_sum = np.sum(resoconto['Hopping'])
  jogging_sum = np.sum(resoconto['Jogging'])
  sitting_sum = np.sum(resoconto['Sitting'])
  walking_sum = np.sum(resoconto['Walking'])

  tot = falling_sum+hopping_sum+jogging_sum+sitting_sum+walking_sum

  resoconto.at[row,'NameActivity'] = 'Statistica'
  resoconto.at[row,"Sitting"] = (sitting_sum/tot)*100
  resoconto.at[row,"Jogging"] = (jogging_sum/tot)*100
  resoconto.at[row,"Fall"] = (falling_sum/tot)*100
  resoconto.at[row,"Hopping"] = (hopping_sum/tot)*100
  resoconto.at[row,"Walking"] = (walking_sum/tot)*100

crea nuova dataframe poi portato a csv, contenente tante attività quante sono state quelle del questionario svolte e per ognuna viene riportato il numero di attività predette

In [ ]:
def count_activities(data):
    counts = Counter(data)
    labels, values = zip(*counts.items())
    return labels, values

def calculate_reso(i, title, labels, values, resoconto):
    resoconto.at[i, 'NameActivity'] = title
    for j in range(len(labels)):
        if labels[j] in ["Fall", "Hopping", "Jogging", "Sitting", "Walking"]:
            resoconto.at[i, labels[j]] = values[j]

def total_activity(file_name, predicted_user, interval):
  resoconto = pd.DataFrame(columns = ["NameActivity","Sitting", "Jogging", "Fall", "Hopping", "Walking"])
  if(len(interval)>0):
    temp = predicted_user['PredictedActivity'].iloc[0:interval[0]].values
    title = predicted_user['NameActivity'].iloc[0]

    labels, values = count_activities(temp)

    indSort = np.argsort(values)[::-1]

    calculate_reso(0, title, labels, values, resoconto)

    i=0
    for i in range(1,len(interval)-1):
      temp = predicted_user['PredictedActivity'].iloc[interval[i]:interval[i+1]].values
      title = predicted_user['NameActivity'].iloc[interval[i]]

      labels, values = count_activities(temp)

      indSort = np.argsort(values)[::-1]

      calculate_reso(i, title, labels, values, resoconto)

    percentage(i+1, resoconto)
  else:
    temp = predicted_user['PredictedActivity'].values
    title = predicted_user['NameActivity'].iloc[0]

    labels, values = count_activities(temp)

    indSort = np.argsort(values)[::-1]

    calculate_reso(0, title, labels, values, resoconto)

    percentage(1, resoconto)

  w1,w2 = file_name.split('.')
  if(not os.path.isdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/reso')):
    os.mkdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/reso')
  resoconto.to_csv('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/reso/%s.csv' % w1)

In [ ]:
for file in os.listdir('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/csv_estratti'):#file=nomefile

  if(not os.path.exists('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/1B/reso/%s' % file)):
    print(file)
    df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/mmsa/Test1/csv_estratti/%s' % file)
    if(df.shape[0]>200):
      df_predicted, predicted_vett = predict(file)
      print('\n')
      translated_vett = translate(predicted_vett)
      print('\n')
      printer(df_predicted)
      print('\n')
      predicted_user = save2(file, translated_vett)
      print('\n')
      interval_activity = find_activity(predicted_user)
      print('\n')
      total_activity(file, predicted_user, interval_activity)
      print('\n\n\n')
    else:
      print(df.shape)
      print('Troppo piccolo')
  else:
    print('File %s già creato' % file)

File __sensor_1674550171975985741_ACCELEROMETER.csv già creato
File __sensor_1674555977600988584_ACCELEROMETER.csv già creato
File __sensor_1674556179331507437_ACCELEROMETER.csv già creato
File __sensor_1674555940422262815_ACCELEROMETER.csv già creato
File __sensor_1674555933384001637_ACCELEROMETER.csv già creato
File __sensor_1674555925901292275_ACCELEROMETER.csv già creato
File __sensor_1674555882761638634_ACCELEROMETER.csv già creato
File __sensor_1674550177359106513_ACCELEROMETER.csv già creato
File __sensor_1674556322315408421_ACCELEROMETER.csv già creato
File __sensor_1674549984991646614_ACCELEROMETER.csv già creato
File __sensor_1674549989677871660_ACCELEROMETER.csv già creato
File __sensor_1674549994110918405_ACCELEROMETER.csv già creato
File __sensor_1674550042014647786_ACCELEROMETER.csv già creato
File __sensor_1674550007826776840_ACCELEROMETER.csv già creato
File __sensor_1674550079987341326_ACCELEROMETER.csv già creato
File __sensor_1674550061542050302_ACCELEROMETER.csv già

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


3994/3994 [==============================] - 98s 25ms/step


[2 2 2 ... 0 0 0]


         Sitting       Jogging          Fall       Hopping       Walking
0       0.000003  2.227869e-06  9.999946e-01  6.972000e-12  4.095322e-08
1       0.000002  6.448444e-06  9.999912e-01  7.754601e-13  1.584047e-07
2       0.000004  6.976737e-06  9.999889e-01  2.138793e-13  4.182961e-07
3       0.000007  6.337003e-05  9.999261e-01  1.189964e-13  3.022811e-06
4       0.000024  5.410084e-04  9.993746e-01  5.706557e-14  6.016946e-05
...          ...           ...           ...           ...           ...
127798  1.000000  1.503077e-11  9.062198e-10  8.296467e-14  1.886435e-12
127799  1.000000  6.727886e-12  1.893206e-10  1.712752e-14  7.540795e-13
127800  1.000000  2.698501e-12  5.636082e-11  1.423952e-14  2.413140e-13
127801  1.000000  2.961079e-12  6.432753e-11  5.888219e-14  1.767503e-13
127802  1.000000  3.709211e-12  1.206047e-10  2.469387e-13  2.183657e-13

[127803 rows x 5 columns]
113741.7
3730.45

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


1699/1699 [==============================] - 42s 25ms/step


[0 0 0 ... 0 0 0]


        Sitting       Jogging          Fall       Hopping       Walking
0      1.000000  1.077104e-29  1.488980e-20  9.334919e-20  2.077265e-23
1      1.000000  3.667212e-28  8.012038e-20  1.288817e-19  1.550019e-22
2      1.000000  1.333101e-26  2.661972e-18  9.215826e-19  9.121518e-22
3      1.000000  1.578757e-24  1.301646e-16  3.086375e-18  2.132931e-20
4      1.000000  2.065554e-23  4.602834e-15  1.057043e-18  5.096454e-20
...         ...           ...           ...           ...           ...
54337  0.995980  2.473254e-03  1.725128e-05  1.513232e-04  1.377746e-03
54338  0.995811  2.598753e-03  1.787574e-05  1.605647e-04  1.412100e-03
54339  0.995998  2.490010e-03  1.744431e-05  1.615557e-04  1.333041e-03
54340  0.996512  2.125565e-03  1.657938e-05  1.451447e-04  1.200500e-03
54341  0.996623  2.051404e-03  1.647650e-05  1.412116e-04  1.167499e-03

[54342 rows x 5 columns]
49196.94
1621.579
2385.4197
4

...

2741/2741 [==============================] - 70s 25ms/step


[0 0 0 ... 2 2 2]


        Sitting       Jogging      Fall       Hopping       Walking
0      0.999564  1.432882e-09  0.000435  5.681521e-09  2.455148e-07
1      0.999625  1.816588e-09  0.000375  9.142716e-09  2.953756e-07
2      0.999738  1.775321e-09  0.000262  1.060465e-08  1.785136e-07
3      0.999751  1.794977e-09  0.000248  7.631892e-09  1.893987e-07
4      0.999794  2.571595e-09  0.000205  1.029098e-08  3.100196e-07
...         ...           ...       ...           ...           ...
87703  0.000056  1.872103e-27  0.999944  3.840041e-22  1.862424e-33
87704  0.000206  1.590725e-26  0.999794  8.415618e-22  1.755260e-33
87705  0.051607  3.532530e-27  0.948394  4.646784e-22  1.937343e-35
87706  0.002445  1.167464e-26  0.997555  2.327015e-22  9.614692e-35
87707  0.000333  1.769381e-26  0.999667  1.630461e-23  5.504773e-35

[87708 rows x 5 columns]
69618.695
2802.0378
8137.237
2589.3206
4560.709
87708.0










__sensor_16

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


1966/1966 [==============================] - 50s 25ms/step


[0 0 0 ... 0 0 0]


        Sitting   Jogging          Fall       Hopping   Walking
0      1.000000  0.000000  8.489665e-33  5.887616e-35  0.000000
1      1.000000  0.000000  5.656631e-33  2.629030e-35  0.000000
2      1.000000  0.000000  3.349548e-35  2.527856e-38  0.000000
3      1.000000  0.000000  1.210398e-35  1.592804e-38  0.000000
4      1.000000  0.000000  2.650710e-36  1.223208e-37  0.000000
...         ...       ...           ...           ...       ...
62886  0.997558  0.000048  2.658424e-04  4.430052e-04  0.001686
62887  0.997399  0.000087  3.588108e-04  5.971218e-04  0.001558
62888  0.996969  0.000082  6.598015e-04  8.020580e-04  0.001487
62889  0.997603  0.000029  7.396404e-04  8.532842e-04  0.000776
62890  0.998446  0.000019  3.818963e-04  7.174952e-04  0.000436

[62891 rows x 5 columns]
46029.12
1680.9088
7755.589
3095.3076
4330.0723
62891.0










__sensor_1674551240391181800_ACCELEROMETER.csv
(90, 5)
Trop

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


29/29 [==============================] - 1s 24ms/step


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2
 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2